# Spin Simulation Example

Construct a system of $N$ identical spins with random dipolar coupling strengths, and simulate their evolution in time.

TODO write out the relevant physics here...

In [ ]:
import spinSimulation as ss
import numpy as np
import importlib

In [ ]:
importlib.reload(ss)

System parameters defined below.

In [ ]:
Ncyc = 10         # number of cycles to simulate
N = 4             # number of spins in system
dim = 2**N        # dimension of system (2^N for spin-1/2)

pulse = .25e-6    # duration of pulse
delay = 3e-6      # duration of delay
f1 = np.pi/(2*pulse)  # for pi/2 pulses
coupling = 2*np.pi * 5e3    # coupling strength
Delta = 2*np.pi * 500       # chemical shift strength (for identical spins)

In [ ]:
a = ss.getRandDip(N) # random dipolar coupling strengths
(x,y,z) = (ss.x, ss.y, ss.z)
(X,Y,Z) = ss.get_total_spin(N, dim)

Hdip = ss.getHdip(N, dim, x, y, z, a)
Hint = ss.getHint(Hdip, coupling, Z, Delta)
HWHH0 = ss.get_H_WHH_0(N, dim, delta)

UWHH0 = ss.get_propagator(HWHH0, 6*delay + 4*pulse)

In [ ]:
Utau   = ss.get_propagator(Hint, delay)
Ux     = ss.get_propagator(Hint + f1*X, pulse)
Uy     = ss.get_propagator(Hint + f1*Y, pulse)
Uxbar  = ss.get_propagator(Hint - f1*X, pulse)
Uybar  = ss.get_propagator(Hint - f1*Y, pulse)

# TODO also do delta-pulses

UWHH = Utau @ Ux @ Utau @ Uybar @ Utau @ Utau @ \
        Uy @ Utau @ Uxbar @ Utau

In [ ]:
rho0 = np.copy(X)    # finite pulse width
rho1 = np.copy(rho0) # zeroth order AHT approximation
rhoinit = np.copy(rho0)
normD = np.trace(rho0 @ rho0)

sig0 = np.zeros((Ncyc, 1), dtype="complex64")
sig1 = np.zeros_like(sig0)
# fid0 = np.copy(sig0)
# fid1 = np.copy(sig0)

for n in range(Ncyc):
    # evolution of collective magnetization
    sig0[n] = np.trace(rho0 @ rhoinit) / normD
    sig1[n] = np.trace(rho1 @ rhoinit) / normD
    
    # time evolution
    rho0 = UWHH  @ rho0 @ UWHH.T.conj()
    rho1 = UWHH0 @ rho1 @ UWHH0.T.conj()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(range(Ncyc), np.real(sig0), label='0')
plt.plot(range(Ncyc), np.real(sig1), label='1')
plt.title('Real component of signal')
plt.xlabel('Cycle number')
plt.ylabel('Net magnetization, real')
plt.legend()